In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime

In [ ]:
dateparse = lambda x: pd.datetime.strptime(x, '%d/%m/%Y') # 03/01/2018
df = pd.read_csv('data/valeursfoncieres-clean-df.csv', parse_dates=['date_mutation'])

In [ ]:
df.info()

In [ ]:
from geopy.exc import GeocoderTimedOut
from geopy.geocoders import GoogleV3

cache_file = 'data/cache.dat'

def save_locations(df):
    df.to_csv(cache_file, index = None, header=True)

def load_locations():
    try:
        return pd.read_csv(cache_file)
    except:
        print('{} doesn\'t exist or is empty - creating a new dataframe'.format(cache_file))
        return pd.DataFrame(columns=['latitude','longitude','address', 'address_unformatted'])

with open('google_api_key.txt') as f:
    google_api_key = f.readline()

def add_locations(df, location, address):
    return df.append({'latitude':location.latitude,
               'longitude':location.longitude,
               'address':location.address,
                'address_unformatted':address},
              ignore_index=True)
    
def get_geocode(df, address):
    location = df[df['address_unformatted']==address]
    if not location.empty :
        return df, location
    #print('Address not in cache, requesting GeoPy')
    geopy = GoogleV3(api_key=google_api_key)
    try:
        location = geopy.geocode(address)
        df = add_locations(df, location, address)
    except Exception as e:
        print('Exception on address: {}'.format(address))
        traceback.print_exc()
    return df, location

In [ ]:
if add_lat_long: 
    df['latitude'] = np.nan
    df.latitude = df.latitude.astype('float')
    df['longitude'] = np.nan
    df.longitude = df.longitude.astype('float')
    df['address_formatted'] = np.nan

In [ ]:
if add_lat_long:
    for index, row in df.iterrows():
        address = row['address']
        #print('Searching address:{}'.format(row['address']))
        geolocation_df, location = get_geocode(geolocation_df, address)
        try:
            if location is not None:
                df.at[index, 'latitude'] = location.latitude
                df.at[index, 'longitude'] = location.longitude
                df.at[index, 'address_formatted'] = location.address
        except Exception as e:
            print('Exception on address [{}]: {}'.format(index, row['address']))
            traceback.print_exc()
        if index%100==0:
            print('Saving - index: {}'.format(index))
            save_locations(geolocation_df)